# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import CNN6AutoEncoder as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/CONV_AE_revised/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
            window_size=config['preprocessing']['window_size'],
            hop_size=config['preprocessing']['hop_size'],
            mel_bins=config['preprocessing']['mel_bins'],
            fmin=config['preprocessing']['fmin'],
            fmax=config['preprocessing']['fmax'])
    pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters())
    criterion = nn.MSELoss()
    num_epochs = config['fit']['num_epochs']
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, num_epochs, writer)
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [11]:
for machine_type in machine_types:
    #if (machine_type == machine_types[0]) or (machine_type == machine_types[1]):
    #    continue
    history = run(machine_type)
    with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
        pickle.dump(history , file)

2020-10-31 23:29:26,882 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-10-31 23:29:26,883 - 00_train.py - INFO - MAKE DATA_LOADER
2020-10-31 23:29:27,709 - 00_train.py - INFO - TRAINING
  0%|          | 0/204 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 1803/1803 [00:05<00:00, 318.93it/s]
2020-10-31 23:30:04,700 - pytorch_modeler.py - INFO - Epoch 1/100:train_loss:722.319673, valid_AUC:0.517213, valid_pAUC:0.504905
100%|██████████| 1803/1803 [00:05<00:00, 318.26it/s]
2020-10-31 23:30:39,251 - pytorch_modeler.py - INFO - Epoch 2/100:train_loss:203.275970, valid_AUC:0.508648, valid_pAUC:0.508244
100%|██████████| 1803/1803 [00:05<00:00, 304.05it/s]
2020-10-31 23:31:14,848 - pytorch_modeler.py - INFO - Epoch 3/100:train_loss:23.201906, valid_AUC:0.489383, valid_pAUC:0.515287
100%|██████████| 1803/1803 [00:05<00:00, 316.52it/s]
2020-10-31 23:31:50,287 - pytorch_modeler.py - INFO - Epoch 4/100:train_loss:9.285129, valid_AUC:0.501254, valid_pAUC:0.517331
100%|██████████| 1803/1803 [00:05<00:00, 311.72it/s]
2020-10-31 23:32:25,326 - pytorch_modeler.py - INFO - Epoch 5/100:train_loss:8.700532, valid_AUC:0.511566, valid_pAUC:0.514086
100%|██████████| 1803/1803 [00:05<00:00, 310.80it/s]
2020-10-31 23:33:01,184 - pytorch_modeler.

100%|██████████| 1803/1803 [00:05<00:00, 310.71it/s]
2020-10-31 23:51:51,556 - pytorch_modeler.py - INFO - Epoch 37/100:train_loss:6.701350, valid_AUC:0.549752, valid_pAUC:0.501894
100%|██████████| 1803/1803 [00:06<00:00, 291.46it/s]
2020-10-31 23:52:28,584 - pytorch_modeler.py - INFO - Epoch 38/100:train_loss:6.691218, valid_AUC:0.553906, valid_pAUC:0.503864
100%|██████████| 1803/1803 [00:10<00:00, 175.53it/s]
2020-10-31 23:53:10,692 - pytorch_modeler.py - INFO - Epoch 39/100:train_loss:6.675260, valid_AUC:0.543921, valid_pAUC:0.502935
100%|██████████| 1803/1803 [00:09<00:00, 192.59it/s]
2020-10-31 23:54:06,823 - pytorch_modeler.py - INFO - Epoch 40/100:train_loss:6.659580, valid_AUC:0.550114, valid_pAUC:0.502570
100%|██████████| 1803/1803 [00:10<00:00, 180.26it/s]
2020-10-31 23:55:03,103 - pytorch_modeler.py - INFO - Epoch 41/100:train_loss:6.644124, valid_AUC:0.556252, valid_pAUC:0.503798
100%|██████████| 1803/1803 [00:09<00:00, 193.82it/s]
2020-10-31 23:55:58,917 - pytorch_modeler.

100%|██████████| 1803/1803 [00:05<00:00, 323.13it/s]
2020-11-01 00:18:25,901 - pytorch_modeler.py - INFO - Epoch 73/100:train_loss:6.361045, valid_AUC:0.552749, valid_pAUC:0.504080
100%|██████████| 1803/1803 [00:05<00:00, 316.95it/s]
2020-11-01 00:19:01,236 - pytorch_modeler.py - INFO - Epoch 74/100:train_loss:6.347972, valid_AUC:0.553999, valid_pAUC:0.503592
100%|██████████| 1803/1803 [00:05<00:00, 307.15it/s]
2020-11-01 00:19:37,177 - pytorch_modeler.py - INFO - Epoch 75/100:train_loss:6.335175, valid_AUC:0.548926, valid_pAUC:0.501679
100%|██████████| 1803/1803 [00:09<00:00, 194.35it/s]
2020-11-01 00:20:25,040 - pytorch_modeler.py - INFO - Epoch 76/100:train_loss:6.349679, valid_AUC:0.551105, valid_pAUC:0.503010
100%|██████████| 1803/1803 [00:09<00:00, 192.85it/s]
2020-11-01 00:21:19,986 - pytorch_modeler.py - INFO - Epoch 77/100:train_loss:6.334860, valid_AUC:0.545978, valid_pAUC:0.502964
100%|██████████| 1803/1803 [00:07<00:00, 235.53it/s]
2020-11-01 00:22:13,126 - pytorch_modeler.

...

use: cuda:0


100%|██████████| 855/855 [00:07<00:00, 119.13it/s]
2020-11-01 08:46:19,990 - pytorch_modeler.py - INFO - Epoch 1/100:train_loss:810.909077, valid_AUC:0.478648, valid_pAUC:0.504020
100%|██████████| 855/855 [00:04<00:00, 196.21it/s]
2020-11-01 08:47:12,444 - pytorch_modeler.py - INFO - Epoch 2/100:train_loss:294.929287, valid_AUC:0.465588, valid_pAUC:0.498699
100%|██████████| 855/855 [00:04<00:00, 195.11it/s]
2020-11-01 08:47:57,000 - pytorch_modeler.py - INFO - Epoch 3/100:train_loss:51.006836, valid_AUC:0.435005, valid_pAUC:0.497802
100%|██████████| 855/855 [00:04<00:00, 195.81it/s]
2020-11-01 08:48:41,760 - pytorch_modeler.py - INFO - Epoch 4/100:train_loss:15.401608, valid_AUC:0.433632, valid_pAUC:0.498901
100%|██████████| 855/855 [00:04<00:00, 194.91it/s]
2020-11-01 08:49:26,703 - pytorch_modeler.py - INFO - Epoch 5/100:train_loss:13.350951, valid_AUC:0.436000, valid_pAUC:0.498554
100%|██████████| 855/855 [00:04<00:00, 194.76it/s]
2020-11-01 08:50:11,405 - pytorch_modeler.py - INFO 

100%|██████████| 855/855 [00:04<00:00, 196.24it/s]
2020-11-01 09:13:16,341 - pytorch_modeler.py - INFO - Epoch 37/100:train_loss:7.435033, valid_AUC:0.539302, valid_pAUC:0.503094
100%|██████████| 855/855 [00:04<00:00, 192.22it/s]
2020-11-01 09:14:01,367 - pytorch_modeler.py - INFO - Epoch 38/100:train_loss:7.412129, valid_AUC:0.541912, valid_pAUC:0.504656
100%|██████████| 855/855 [00:04<00:00, 191.99it/s]
2020-11-01 09:14:45,852 - pytorch_modeler.py - INFO - Epoch 39/100:train_loss:7.388311, valid_AUC:0.538698, valid_pAUC:0.503528
100%|██████████| 855/855 [00:04<00:00, 192.51it/s]
2020-11-01 09:15:30,186 - pytorch_modeler.py - INFO - Epoch 40/100:train_loss:7.346440, valid_AUC:0.541055, valid_pAUC:0.504627
100%|██████████| 855/855 [00:04<00:00, 195.33it/s]
2020-11-01 09:16:15,044 - pytorch_modeler.py - INFO - Epoch 41/100:train_loss:7.335320, valid_AUC:0.540275, valid_pAUC:0.503846
100%|██████████| 855/855 [00:04<00:00, 195.42it/s]
2020-11-01 09:16:59,920 - pytorch_modeler.py - INFO - 

100%|██████████| 855/855 [00:04<00:00, 190.91it/s]
2020-11-01 09:40:19,906 - pytorch_modeler.py - INFO - Epoch 73/100:train_loss:6.874139, valid_AUC:0.559495, valid_pAUC:0.499711
100%|██████████| 855/855 [00:02<00:00, 325.70it/s]
2020-11-01 09:40:56,994 - pytorch_modeler.py - INFO - Epoch 74/100:train_loss:6.881932, valid_AUC:0.560368, valid_pAUC:0.500405
100%|██████████| 855/855 [00:02<00:00, 329.18it/s]
2020-11-01 09:41:25,275 - pytorch_modeler.py - INFO - Epoch 75/100:train_loss:6.848237, valid_AUC:0.559769, valid_pAUC:0.503702
100%|██████████| 855/855 [00:02<00:00, 326.21it/s]
2020-11-01 09:41:53,651 - pytorch_modeler.py - INFO - Epoch 76/100:train_loss:6.834808, valid_AUC:0.562231, valid_pAUC:0.498554
100%|██████████| 855/855 [00:02<00:00, 322.38it/s]
2020-11-01 09:42:22,023 - pytorch_modeler.py - INFO - Epoch 77/100:train_loss:6.824500, valid_AUC:0.562280, valid_pAUC:0.502458
100%|██████████| 855/855 [00:02<00:00, 325.71it/s]
2020-11-01 09:42:50,344 - pytorch_modeler.py - INFO - 

elapsed time: 4075.035981655 [sec]


history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)

In [12]:
history['reconstruct_img']['label']

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0)]